In [1]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import sys
sys.path.append("..") 
from Portfolio.portfolio_tools import *

import tink_port as tink

Вводим токен, получаем список доступных аккаунтов

In [2]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'

token = TOKEN0

In [3]:
accs = tink.get_accounts(token)
print("Количество аккаунтов:", len(accs.accounts))
accs.accounts[0].name

Количество аккаунтов: 1


'Брокерский счёт'

#### База идентификаторов Tinkoff

In [4]:
base = tink.get_id_base(token)
port = tink.get_portfolio(token)
df_port = tink.port_to_df(port, base)

In [5]:
df_port

,figi,ticker,name,quantity,price
2,BBG004S68758,BANE,Башнефть,14,3159.000
10,BBG00475K6C3,CHMF,Северсталь,30,1865.800
5,BBG012YQ6P43,CIAN,Циан АДР,81,872.000
0,BBG004731032,LKOH,ЛУКОЙЛ,7,7654.000
4,BBG00Y91R9T3,OZON,Ozon Holdings PLC,23,3845.500
9,TCS00A103X66,POSI,Positive Technologies,35,2921.200
1,BBG004S682Z6,RTKM,Ростелеком,840,100.930
6,BBG004S681M2,SNGSP,Сургутнефтегаз - привилегированные акции,900,66.815
8,TCS00A105EX7,WUSH,Whoosh,324,311.200
3,BBG006L8G4H1,YNDX,Яндекс,15,3868.000


### Load account data

In [93]:
res = []

for pos in port.positions:
    print(pos.figi)        
    candles = tink.get_candles(token, pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  tink.get_open_price(candles)
    ticker = tink.figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)

BBG004731032
BBG004S681B4
TCS00A107RZ0
BBG007N0Z367
BBG00475KHX6
BBG004S68507
BBG004RVFFC0
BBG004S684M6
BBG004S686N0
BBG004S681M2
RUB000UTSTOM
TCS00A103X66
BBG004730N88
BBG00475K6C3
BBG004731354


In [94]:
dfp = dfp.dropna(axis = 1)
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


### Portfolio Black-Litterman Model

In [8]:
mcaps = {}
for t in dfp.columns:
   # mcaps[t] = stock.info["marketCap"]
    mcaps[t] = 1
mcaps

{'LKOH': 1,
 'NLMK': 1,
 'AGRO': 1,
 'MAGN': 1,
 'TATN': 1,
 'SIBN': 1,
 'BANEP': 1,
 'SNGSP': 1,
 'POSI': 1,
 'SBER': 1,
 'CHMF': 1,
 'ROSN': 1}

In [9]:
mcaps['SBER'] = 1.5
mcaps['POSI'] = 1.5

In [28]:
imoex_figi = 'BBG333333333'
market_prices = tink.get_candles(token, imoex_figi, CandleInterval.CANDLE_INTERVAL_DAY)
market_prices =  tink.get_open_price(market_prices)

In [21]:
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


In [38]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel

S = risk_models.CovarianceShrinkage(dfp).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices['ticker'])
delta

9.802403162030824

In [39]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

LKOH     0.235540
NLMK     0.250621
AGRO     0.404671
MAGN     0.222163
TATN     0.175982
SIBN     0.177426
BANEP    0.306354
SNGSP    0.265854
POSI     0.254061
SBER     0.201803
CHMF     0.241332
ROSN     0.239506
dtype: float64

In [41]:
# You don't have to provide views on all the assets
viewdict = {
    "POSI": 0.10,
}

confidences = [
    0.6
]
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

In [43]:
intervals = [
    (0.1, 0.4),
]

variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

[0.022500000000000006]


In [44]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [45]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl

LKOH     0.210907
NLMK     0.225580
AGRO     0.377385
MAGN     0.198196
TATN     0.153239
SIBN     0.155086
BANEP    0.281036
SNGSP    0.241803
POSI     0.213216
SBER     0.178689
CHMF     0.218008
ROSN     0.216051
dtype: float64

In [46]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
LKOH,0.235540,0.210907,NaN
NLMK,0.250621,0.225580,NaN
AGRO,0.404671,0.377385,NaN
MAGN,0.222163,0.198196,NaN
TATN,0.175982,0.153239,NaN
SIBN,0.177426,0.155086,NaN
BANEP,0.306354,0.281036,NaN
SNGSP,0.265854,0.241803,NaN
POSI,0.254061,0.213216,0.1
SBER,0.201803,0.178689,NaN


In [49]:
S_bl = bl.bl_cov()


In [50]:
from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights
     

D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


OrderedDict([('LKOH', 0.07897),
             ('NLMK', 0.08567),
             ('AGRO', 0.14173),
             ('MAGN', 0.07393),
             ('TATN', 0.05579),
             ('SIBN', 0.0562),
             ('BANEP', 0.10686),
             ('SNGSP', 0.09078),
             ('POSI', 0.08075),
             ('SBER', 0.06611),
             ('CHMF', 0.08197),
             ('ROSN', 0.08124)])

In [52]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, dfp.iloc[-1], total_portfolio_value=100000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

Leftover: $703.29


D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


{'LKOH': 1,
 'NLMK': 39,
 'AGRO': 10,
 'MAGN': 130,
 'TATN': 8,
 'SIBN': 8,
 'BANEP': 5,
 'SNGSP': 141,
 'POSI': 3,
 'SBER': 22,
 'CHMF': 4,
 'ROSN': 15}

### Model

In [56]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

## All shares

In [6]:
dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]

In [8]:
res = []
for row in dfx.iterrows():
    pos = row[1]
#    print(pos.figi)        
    candles = tink.get_candles(token, pos.figi, CandleInterval.CANDLE_INTERVAL_DAY, 50)
    df =  tink.get_open_price(candles)
    ticker = tink.figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
df_full = pd.concat(res, axis = 1)


In [42]:
dfp = df_full.dropna(axis = 1).copy()
dfp

,KZIZP,IRKT,VSMO,UNAC,VKCO,TTLK,MGNT,SPBE,SVCB,KZIZ,...,AFKS,PLZL,LKOH,DELI,MGKL,NMTP,TGKA,CNTLP,MTLRP,MRKY
date,,,,,,,,,,,,,,,,,,,,,
2024-02-13,609.34,88.65,40020.0,1.5890,672.6,1.0800,7249.5,91.1,17.250,798.37,...,17.946,11102.0,7297.0,270.30,2.3650,12.575,0.009850,12.52,401.00,0.10690
2024-02-14,557.07,90.30,40220.0,1.5810,693.8,1.0870,7270.0,95.0,17.900,779.50,...,18.121,11080.0,7275.0,266.95,2.3900,12.800,0.010030,12.36,401.75,0.10645
2024-02-15,533.08,86.80,40280.0,1.5605,681.0,1.1030,7420.5,93.4,17.585,775.00,...,18.033,11029.0,7205.0,282.00,2.3875,12.645,0.009910,12.46,401.50,0.10475
2024-02-16,550.00,88.50,40100.0,1.5730,677.8,1.1255,7484.5,93.2,17.600,782.21,...,18.220,11040.0,7294.0,281.00,2.3820,13.005,0.009932,12.50,399.40,0.10290
2024-02-19,559.00,85.70,39260.0,1.5430,660.0,1.1080,7438.0,91.5,18.575,762.46,...,18.150,10851.0,7233.0,280.15,2.3775,12.890,0.009798,12.10,399.50,0.10210
2024-02-20,546.44,85.75,39340.0,1.5335,660.4,1.1010,7399.5,92.0,18.865,852.21,...,17.893,10951.0,7188.0,287.75,2.4400,13.250,0.009788,12.24,390.90,0.10170
2024-02-21,506.19,80.00,38200.0,1.5095,644.0,1.0680,7254.0,88.5,18.200,748.19,...,17.500,10704.5,7080.0,282.00,2.3960,12.475,0.009598,11.96,385.95,0.10105
2024-02-22,510.00,78.20,36840.0,1.5290,642.0,1.0000,7210.0,86.5,18.200,734.38,...,17.601,10728.0,7000.5,274.05,2.3790,12.020,0.009392,11.40,353.00,0.09795
2024-02-26,540.85,80.20,37020.0,1.5250,653.0,1.0730,7344.0,84.0,18.300,730.93,...,18.000,10899.0,7150.5,275.00,2.3585,12.465,0.009340,11.72,343.90,0.09490


In [87]:
drops = ["GTRK", "SFIN", "ORUP", "UWGN", "RKKE", "KROT", "APTK", "VEON-RX"]
drops = ["SFIN", "GTRK", "NTZL", "LSRG", 'TGKBP', 'FIVE']
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

In [59]:
drops = ['FIVE']
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

index_assets = pd.read_csv('index_assets.csv')['asset'].values.tolist()
columns = [x for x in dfp.columns if x in index_assets]
dfp = dfp[columns]
columns

['WUSH',
 'MAGN',
 'BANEP',
 'TATN',
 'RTKM',
 'SNGSP',
 'NVTK',
 'AQUA',
 'OZON',
 'ROSN',
 'CHMF',
 'SMLT',
 'BANE',
 'CIAN',
 'SBER',
 'POSI',
 'SIBN',
 'NLMK',
 'YNDX',
 'LKOH']

### RiskFolio

In [60]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

lookback = 30
df_period = dfp[-lookback:]

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 10
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

#print_data(dfp)
df_period = dfp.dropna(axis = 1)

dfw = riskfolio_weights(df_period, 'CVaR', 'MaxRet')
dfw

,weights
WUSH,0.105214614
RTKM,0.152706069
SNGSP,0.068692562
AQUA,0.032449822
OZON,0.121548309
CHMF,0.067025377
BANE,0.071536810
CIAN,0.104990284
SBER,0.010721283
POSI,0.124119764


In [87]:
dfx = final_sums(dfw, 783775, 50)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

inds
x = base[base['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s

In [88]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["lot_quantity"] = np.round(dfx.weights/ (dfx.price * dfx.lot))
dfx["quantity"] =  dfx.lot * dfx.lot_quantity
dfx["sum"]= dfx.price * dfx.lot_quantity * dfx.lot
dfx = dfx.sort_values("weights", ascending = False)
dfx.to_csv("t.csv")
dfx = dfx.reset_index()
dfx.columns = ['ticker', 'weights', 'lot', 'price', 'lot_quantity','quantity', 'sum']

dfx

,ticker,weights,lot,price,lot_quantity,quantity,sum
0,RTKM,119690.0,10,103.050,116.0,1160.0,119538.0
1,POSI,97280.0,1,2912.000,33.0,33.0,96096.0
2,OZON,95270.0,1,3875.000,25.0,25.0,96875.0
3,WUSH,82460.0,1,318.000,259.0,259.0,82362.0
4,CIAN,82290.0,1,880.000,94.0,94.0,82720.0
5,YNDX,66020.0,1,3980.000,17.0,17.0,67660.0
6,BANE,56070.0,1,3200.000,18.0,18.0,57600.0
7,SNGSP,53840.0,100,67.085,8.0,800.0,53668.0
8,CHMF,52530.0,1,1857.400,28.0,28.0,52007.2
9,NLMK,31300.0,10,219.000,14.0,140.0,30660.0


In [89]:
df_port

,figi,ticker,name,quantity,price
2,BBG004S68758,BANE,Башнефть,14,3159.000
10,BBG00475K6C3,CHMF,Северсталь,30,1865.800
5,BBG012YQ6P43,CIAN,Циан АДР,81,872.000
0,BBG004731032,LKOH,ЛУКОЙЛ,7,7654.000
4,BBG00Y91R9T3,OZON,Ozon Holdings PLC,23,3845.500
9,TCS00A103X66,POSI,Positive Technologies,35,2921.200
1,BBG004S682Z6,RTKM,Ростелеком,840,100.930
6,BBG004S681M2,SNGSP,Сургутнефтегаз - привилегированные акции,900,66.815
8,TCS00A105EX7,WUSH,Whoosh,324,311.200
3,BBG006L8G4H1,YNDX,Яндекс,15,3868.000


In [90]:
def df_to_dict(dfx):
    return {row.ticker:row.quantity for _,row in dfx.iterrows()}

old_port = df_to_dict(df_port)
new_port = df_to_dict(dfx)
new_port

{'RTKM': 1160.0,
 'POSI': 33.0,
 'OZON': 25.0,
 'WUSH': 259.0,
 'CIAN': 94.0,
 'YNDX': 17.0,
 'BANE': 18.0,
 'SNGSP': 800.0,
 'CHMF': 28.0,
 'NLMK': 140.0,
 'AQUA': 26.0,
 'LKOH': 1.0,
 'SBER': 30.0}

In [91]:
old_port

{'BANE': 14,
 'CHMF': 30,
 'CIAN': 81,
 'LKOH': 7,
 'OZON': 23,
 'POSI': 35,
 'RTKM': 840,
 'SNGSP': 900,
 'WUSH': 324,
 'YNDX': 15,
 None: 64913}

In [92]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]
    # Сортировка по значению, по возрастанию
    sorted_diff = sorted(difference.items(), key=lambda x: x[1])

    return sorted_diff

In [93]:
res = calculate_portfolio_difference(old_port, new_port)
res

[(None, -64913),
 ('SNGSP', -100.0),
 ('WUSH', -65.0),
 ('LKOH', -6.0),
 ('POSI', -2.0),
 ('CHMF', -2.0),
 ('OZON', 2.0),
 ('YNDX', 2.0),
 ('BANE', 4.0),
 ('CIAN', 13.0),
 ('AQUA', 26.0),
 ('SBER', 30.0),
 ('NLMK', 140.0),
 ('RTKM', 320.0)]